# Topic 5. Recursive functions and data types

In [ ]:
def proof[A, B]: (P => Q) => (Q => R) => P => R = 
    ??? 

The goals of this topic are to understand:

* How recursive types (lists, trees, etc.) are defined algebraically
* How functions over recursive types are defined recursivelly
* The two major types of recursive functions: general and tail-recursive

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 16. Working with Lists
- Chapter 26. Extractors (optional)

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 29-36. Recursion.

## Recursive types

### The `List` type

In [1]:
// [1, 2, 3, 4]
// [123, 22, 55, 322, 22]
// []

// ['a', 'b']

// ['a', 1, 6.7]

In [2]:
val d: Int = 1

d: Int = 1

In [3]:
d

res3: Int = 1

In [3]:
d = 3

-- [E052] Type Error: cmd4.sc:1:13 ---------------------------------------------
1 |val res4 = d = 3
  |           ^^^^^
  |           Reassignment to val d
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [4]:
var f: Int = 0

f: Int = 0

In [5]:
f = 7

In [6]:
f

res6: Int = 7

In [7]:
val l: List[Int] = List(1,2,3,4)

l: List[Int] = List(1, 2, 3, 4)

In [8]:
val empty: List[Int] = List()

empty: List[Int] = List()

In [10]:
val l: List[Int] = 3 :: (5 :: (6 :: Nil))

l: List[Int] = List(3, 5, 6)

In [11]:
val l: List[Int] = ::(3, ::(5, ::(6, Nil)))

l: List[Int] = List(3, 5, 6)

In [12]:
List(1,2,3).::(0)

res12: List[Int] = List(0, 1, 2, 3)

// type List[A] = Nil + A :: List[A]

// type List[A] = 1 + A * List[A]

// type List[A] = 1 + A + A*A + A*A*A + A*A*A*A + ...
//              = 1 + A*(1+ A + A*A + A*A*A + ...)
//              = 1 + A*List[A]

In [22]:
val n: List[Int] = Nil
val l1: List[Int] = 1 :: Nil
val l2: List[Int] = 1 :: 2 :: Nil
// ....


n: List[Int] = List()
l1: List[Int] = List(1)
l2: List[Int] = List(1, 2)

In [23]:
def isEmpty(l: List[Int]): Boolean = 
    l match 
        case Nil => ??? : Boolean
        case head :: tail => ??? : Boolean

defined function isEmpty

In [24]:
def isEmpty(l: List[Int]): Boolean = 
    l match 
        case Nil => true : Boolean
        case head :: tail => false : Boolean

defined function isEmpty

In [26]:
isEmpty(List(1,2,3))
isEmpty(Nil)

res26_0: Boolean = false
res26_1: Boolean = true

In [12]:
// type Either[A, B] = A + B

In [20]:
object Std: 

    enum List[A]: 
        case Nil()
        case NonEmpty(head: A, tail: List[A])

    enum Either[A, B]: 
        case Left[A, B](a: A) extends Either[A, B]
        case Right[A, B](b: B) extends Either[A, B]

    enum Option[A]: 
        case None()
        case Some(a: A)

    case class Tuple2[A, B](a: A, b: B)

defined object Std

In [18]:
val t: Tuple2[Int, String] = Tuple2(1, "S")

t: (Int, String) = (1, "S")

In [19]:
val t2: (Int, String) = (1, "s")

t2: (Int, String) = (1, "s")

In [15]:
val e: Either[Int, String] = Left(1)
val e1: Either[Int, String] = Right("S")

e: Either[Int, String] = Left(value = 1)
e1: Either[Int, String] = Right(value = "S")

In [10]:
val l: List[Int] = 3 :: (5 :: (6 :: Nil))

l: List[Int] = List(3, 5, 6)

In [7]:
l = List(1,2)

-- [E052] Type Error: cmd8.sc:1:13 ---------------------------------------------
1 |val res8 = l = List(1,2)
  |           ^^^^^^^^^^^^^
  |           Reassignment to val l
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [7]:
l.update(3) = 5

-- [E008] Not Found Error: cmd8.sc:1:13 ----------------------------------------
1 |val res8 = l.update(3) = 5
  |           ^^^^^^^^
  |value update is not a member of List[Int], but could be made available as an extension method.
  |
  |The following import might fix the problem:
  |
  |  import ammonite.util.Ref.refer
  |
Compilation Failed

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [1]:
object StdDefinition:


defined object StdDefinition

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [2]:
object ActualStdDefinition:


defined object ActualStdDefinition

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [ ]:
// Less beautifully 

// More idiomatically


And we can also pattern match on lists, similarly:

In [4]:
// Less beautifully


// more idiomatically


// or



res4_0: Int = 1
res4_1: Int = 1
res4_2: Int = 1

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [76]:
// Using mutable variables
def lengthI(l: List[Int]): Int = 
    var sol: Int = ??? 
    var aux: List[Int] = ??? 
    while aux != Nil do 
        sol = ???(sol, aux.head)
        aux = aux.tail
    /*return*/ sol

-- [E050] Type Error: cmd77.sc:5:14 --------------------------------------------
5 |        sol = ???(sol, aux.head)
  |              ^^^
  |              method ??? in object Predef does not take parameters
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [77]:
// Using mutable variables
def lengthI(l: List[Int]): Int = 
    var sol: Int = 0
    var aux: List[Int] = l
    while aux != Nil do 
        sol = sol + 1
        aux = aux.tail
    /*return*/ 
    sol

defined function lengthI

In [11]:
// Using mutable variables
def lengthTR(l: List[Int]): Int = 

    def step(aux: List[Int], sol: Int):  Int = 
        if aux == Nil then sol
        else 
            val nextAux = aux.tail
            val nextSol = sol + 1
            step(nextAux, nextSol)

    step(l, 0)
/*
    var sol: Int = 0
    var aux: List[Int] = l
    while aux != Nil do 
        sol = sol + 1
        aux = aux.tail
    /*return*/ 
    sol*/

defined function lengthTR

In [12]:
// Using mutable variables
def lengthTR(l: List[Int]): Int = 

    def step(aux: List[Int], sol: Int):  Int = 
        aux match 
            case Nil => sol
            case head :: tail => 
                step(aux.tail, sol + 1)

    step(l, 0)
/*
    var sol: Int = 0
    var aux: List[Int] = l
    while aux != Nil do 
        sol = sol + 1
        aux = aux.tail
    /*return*/ 
    sol*/

defined function lengthTR

In [ ]:
aux             sol
---             ---
1::List(2,3)     0
2::List(3)       0+1
3::List()        1+1
List()           2+1
                 

In [77]:
// Using mutable variables
def lengthI(l: List[Int]): Int = 
    var sol: Int = 0
    var aux: List[Int] = l
    while aux != Nil do 
        sol = sol + 1
        aux = aux.tail
    /*return*/ 
    sol

defined function lengthI

In [78]:
lengthI(List(1,2,3,4))

res78: Int = 4

In [6]:
// invoke


res6_0: Int = 0
res6_1: Int = 4

The recursive function is implemented as follows: 

In [31]:
// Using recursive functions
def length(l: List[Int]): Int /* & es la longitud de la lista l*/ = 
    0: Int 

defined function length

In [36]:
// Using recursive functions
def length(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => ??? : Int

defined function length

In [36]:
// Using recursive functions
def length(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = ???
            ??? : Int

defined function length

In [38]:
// Using recursive functions
def length(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = length(tail)
            1+tailLength : Int

defined function length

In [39]:
// Using recursive functions
def length(l: List[Char]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = length(tail)
            1+tailLength : Int

defined function length

In [40]:
// Using recursive functions
def length(l: List[String]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = length(tail)
            1+tailLength : Int

defined function length

In [44]:
// Using recursive functions
def myLength[A](l: List[A]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = length(tail)
            1+tailLength : Int

defined function myLength

In [35]:
def foo[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case (head: A) :: (tail: List[A]) => 
            ??? : B

defined function foo

In [35]:
def foo[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case (head: A) :: (tail: List[A]) => 
            val tailSol: B = foo(tail)
            ??? : B

defined function foo

In [29]:
List(1,2,3).isEmpty

res29: Boolean = false

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that are available, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with very long lists.

### Tail-recursive functions

The implementation using tail-recursion solves the issues with the stack. It commonly makes use of auxiliary functions:

In [8]:
// Using tail-recursive functions



defined function lengthTR

res9_0: Int = 0
res9_1: Int = 3

We can check the stack-safety problems of non-tail recursive functions by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [10]:
// First, imperatively



defined function constantList

In [11]:
// Next, tail-recursively



defined function constantList

We can also use the function [`fill`](https://www.scala-lang.org/api/2.13.3/scala/collection/immutable/List$.html#fill[A](n:Int)(elem:=%3EA):CC[A]) of the Scala standard library.

Now, let's calculate the length of a list long enough to blow up the stack, using each of the three implementations:

In [12]:
// Imperatively


res12: Int = 100000

In [13]:
// Tail-recursive


res13: Int = 100000

In [13]:
// Plain recursive


### Unit testing with `scalatest`

In [5]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. The test catalogue receives the actual function to be tested as a parameter. For instance, this is a possible test class for the `length` function:

In [55]:
// Using recursive functions
def myLength[A](l: List[A]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = length(tail)
            1+tailLength : Int

defined function myLength

In [60]:
class TestLength() extends AnyFlatSpec with should.Matchers:
    "length" should "work for empty list" in:
        myLength(Nil) shouldBe 0

    it should "work for non-empty lists" in:
        myLength[Int](List(1)) shouldBe 1
        myLength(List(1,2,3)) shouldBe 3


defined class TestLength

In [59]:
run(TestLength())

cmd58$Helper$TestLength:
length
- should work for empty list
- should work for non-empty lists


In [ ]:
class TestLength(lengthF: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "length" should "work" in:
        ??? 

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [16]:
run(TestLength(lengthR))

cell15$Helper$TestLength:
length
- should work


### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [61]:
def sum(l: List[Int]): Int = 
    ???

defined function sum

In [63]:
def sum(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => 
            ??? : Int

defined function sum

In [64]:
def sum(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => 
            val tailSum: Int = ??? 
            ??? : Int

defined function sum

In [67]:
def sum(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailSum: Int = ??? 
            ??? : Int

defined function sum

In [4]:
def sumR(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailSum: Int = sumR(tail)
            head+tailSum : Int

defined function sumR

In [ ]:
// Using mutable variables
def addI(l: List[Int]): Int = 
    var sol: Int = ??? 
    var aux: List[Int] = ??? 
    while aux != Nil do 
        sol = ???(sol, aux.head)
        aux = aux.tail
    /*return*/ sol

In [5]:
// Using mutable variables
def addI(l: List[Int]): Int = 
    var sol: Int = 0
    var aux: List[Int] = l 
    while aux != Nil do 
        sol = sol + aux.head
        aux = aux.tail
    /*return*/ sol

defined function addI

In [80]:
addI(List(1,2,3,4))

res80: Int = 10

In [81]:
class TestSumI() extends AnyFlatSpec with should.Matchers:
    "sum" should "work" in:
        addI(List()) shouldBe 0
        addI(List(1)) shouldBe 1
        addI(List(1,2,3,4)) shouldBe 1+(2+(3+4)) // 10

defined class TestSumI

In [82]:
run(TestSumI())

cmd81$Helper$TestSumI:
sum
- should work


In [74]:
class TestSum() extends AnyFlatSpec with should.Matchers:
    "sum" should "work" in:
        sum(List()) shouldBe 0
        sum(List(1)) shouldBe 1
        sum(List(1,2,3,4)) shouldBe 1+(2+(3+4)) // 10

defined class TestSum

In [83]:
class TestSum(sum: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "sum" should "work" in:
        sum(List()) shouldBe 0
        sum(List(1)) shouldBe 1
        sum(List(1,2,3,4)) shouldBe 1+(2+(3+4)) // 10

defined class TestSum

In [85]:
run(TestSum(sumR))

cmd83$Helper$TestSum:
sum
- should work


In [86]:
run(TestSum(addI))

cmd83$Helper$TestSum:
sum
- should work


In [18]:
// Recursively



defined function sum

In [19]:
run(TestSum(sum))

cell17$Helper$TestSum:
sum
- should work


In [20]:
// With tail-recursion



defined function sumTR

In [21]:
run(TestSum(sumTR))

cell17$Helper$TestSum:
sum
- should work


In [ ]:
addI(List.fill(1000000000)(1))

Exception in thread "SIGINT handler" java.lang.UnsupportedOperationException
	at java.base/java.lang.Thread.stop(Thread.java:1667)
	at almond.Execute.interruptible$$anonfun$1(Execute.scala:269)


In [9]:
sumR(List.fill(100000)(1))

java.lang.StackOverflowError: null

In [88]:
List.fill(4)(10)
List.fill(100)(0)

res88_0: List[Int] = List(10, 10, 10, 10)
res88_1: List[Int] = List(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...

### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [22]:
class TestProduct(product: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "product" should "work" in:
        ???

defined class TestProduct

 This is the common recursive implementation:

defined function product

In [24]:
run(TestProduct(product))

cell22$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [25]:
// optimization for 0



defined function product2

In [26]:
run(TestProduct(product2))

cell22$Helper$TestProduct:
product
- should work


A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [27]:
class TestMember(member: (List[Int], Int) => Boolean) extends AnyFlatSpec with should.Matchers:
    "member" should "work" in:
        ???

defined class TestMember

defined function member

In [29]:
run(TestMember(member))

cell27$Helper$TestMember:
member
- should work


We can also pattern match against a specific value as follows:

defined function member

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [31]:
class TestLast(last: List[Int] => Option[Int]) extends AnyFlatSpec with should.Matchers:
    "last" should "work" in:
        ???

defined class TestLast

In [32]:
@annotation.tailrec


defined function last

In [33]:
run(TestLast(last))

cell31$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [47]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "insertLast" should "work" in:
        ???

defined class TestInsertLast

defined function insertLast

In [49]:
run(TestInsertLast(insertLast))

cell47$Helper$TestInsertLast:
insertLast
- should work


### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [8]:
List(3,2).appended(1)

res8: List[Int] = List(3, 2, 1)

In [25]:
class TestReverse(reverse: List[Int] => List[Int]) extends AnyFlatSpec with should.Matchers:
    "reverse" should "work" in:
        reverse(List(1,2,3)) shouldBe List(3,2,1)
        reverse(1 :: List(2,3)) shouldBe List(3,2) :+ 1
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3,2,1)) shouldBe List(1,2,3,2,1)
        reverse(1 :: List(2,3,2,1)) shouldBe {
            val tailReversed = List(1,2,3,2) // List(2,3,2,1).reverse
            tailReversed :+ 1 // List(1,2,3,2,1)
        }
        reverse(List()) shouldBe List()

defined class TestReverse

In [29]:
class TestReverse(reverse: [a] => List[a] => List[a]) extends AnyFlatSpec with should.Matchers:
    "reverse" should "work" in:
        reverse(List('a', 'b', 'c')) shouldBe List('c','b','a')
        reverse(List(1,2,3)) shouldBe List(3,2,1)
        reverse(1 :: List(2,3)) shouldBe List(3,2) :+ 1
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3,2,1)) shouldBe List(1,2,3,2,1)
        reverse(1 :: List(2,3,2,1)) shouldBe {
            val tailReversed = List(1,2,3,2) // List(2,3,2,1).reverse
            tailReversed :+ 1 // List(1,2,3,2,1)
        }
        reverse(List()) shouldBe List()

defined class TestReverse

In [10]:
def reverse(l: List[Int]): List[Int] = 
    l match 
        case Nil => ??? : List[Int]
        case head :: tail => 
            val tailReversed: List[Int] = reverse(tail)
            ??? : List[Int]

defined function reverse

In [10]:
def reverse(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil : List[Int]
        case head :: tail => 
            val tailReversed: List[Int] = reverse(tail)
            ??? : List[Int]

defined function reverse

In [18]:
//@annotation.tailrec
def reverse(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil : List[Int]
        case head :: tail => 
            val tailReversed: List[Int] = reverse(tail)
            tailReversed :+ head : List[Int]

defined function reverse

In [24]:
//@annotation.tailrec
def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => Nil : List[A]
        case head :: tail => 
            val tailReversed: List[A] = reverse(tail)
            tailReversed :+ head : List[A]

defined function reverse

In [30]:
run(TestReverse([a] => l => reverse[a](l)))

cmd29$Helper$TestReverse:
reverse
- should work


In [14]:
reverse(List.fill(10000)(0))

java.lang.StackOverflowError: null

In [ ]:
aux            out
---            --- 
1::List(2,3)   List()
2::List(3)     1::Nil 
3::List()      2::1::Nil
List()         3::2::1::Nil

In [15]:
// Tail-recursive, efficiently
def reverseTR(l: List[Int]): List[Int] = 

    def step(aux: List[Int], out: List[Int]): List[Int] = 
        aux match 
            case Nil => ??? : List[Int]
            case head::tail => 
                val updatedSol: List[Int] = ??? /*(head, out)*/
                step(tail, updatedSol)

    step(l, ???)

defined function reverseTR

In [19]:
// Tail-recursive, efficiently
def reverseTR(l: List[Int]): List[Int] = 

    @annotation.tailrec
    def step(aux: List[Int], out: List[Int]): List[Int] = 
        aux match 
            case Nil => out : List[Int]
            case head::tail => 
                val updatedSol: List[Int] = head :: out /*(head, out)*/
                step(tail, updatedSol)

    step(l, Nil)

defined function reverseTR

In [20]:
run(TestReverse(reverseTR))

cmd6$Helper$TestReverse:
reverse
- should work


In [23]:
reverseTR(List.fill(100000)(0))

res23: List[Int] = List(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...

### Example: concatenate lists

In [56]:
class TestConcatenate(concatenate: (List[Int], List[Int]) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "concatenate" should "work" in:
        ???

defined class TestConcatenate

defined function concatenate

In [58]:
run(TestConcatenate(concatenate))

cell56$Helper$TestConcatenate:
concatenate
- should work


Tail-recursive concatenation:

defined function concatenateTR

In [60]:
run(TestConcatenate(concatenateTR))

cell56$Helper$TestConcatenate:
concatenate
- should work
